<a href="https://colab.research.google.com/github/IgorekFromRU/RecSys/blob/master/Collarobativ_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Коллаборативная фильтрация

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162726 sha256=fa73eff44676cf4eabf3efea2ce05b3bef87532106caa2c2818ec96035d56583
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
from google.colab import drive
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

# Монтируем Google Drive
drive.mount('/content/drive')

# Устанавливаем рабочий каталог
import os
os.chdir('/content/drive/MyDrive/DS/')

# Путь к файлу
file_path = 'data/u.data.txt'

# Загружаем датасет
reader = Reader(line_format="user item rating timestamp", sep="\t")
data = Dataset.load_from_file(file_path, reader=reader)


Mounted at /content/drive


В данных присутствуют следующие признаки:

- userId — идентификаторы пользователей сайта movielens;
- movieId — идентификаторы фильмов;
- rating — оценки фильмов, выставленные пользователями по шкале от 1 до 5;
- timestamp — время оценки фильма пользователем. Данный формат представления времени показывает, сколько секунд прошло с 1 января 1970 года.

In [7]:
import pandas as pd

df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

df.head()

,userId,movieId,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


## Задание 3.1

Сколько уникальных фильмов в наборе данных?

In [8]:
df['movieId'].nunique()

1682

## Задание 3.2

Сколько уникальных пользователей в наборе данных?

In [9]:
df.userId.nunique()

943

## Задание 3.3

Какая оценка встречается в наборе данных чаще всего? Введите ответ в виде целого числа.

In [13]:
df.rating.value_counts().index[0]

4.0

# Задание 3.4

Разбейте данные на обучающую и тестовую выборки. Объём тестовой выборки должен составлять 25 % от общего объёма данных. В качестве значения параметра random_state возьмите число 13.

Сколько объектов попало в тестовую выборку?

In [14]:
from surprise import model_selection

trainset, testset = model_selection.train_test_split(data, test_size=0.25, random_state=13)

print("Количество объектов в тестовой выборке:", len(testset))


Количество объектов в тестовой выборке: 25000


Импортируем функции для построения рекомендательных систем (**SVD** — для model-based-подхода и **KNNBasic** — для memory-basic-подхода) и для оценки качества результата.

In [15]:
from surprise import SVD, KNNBasic, accuracy

Реализуем обычную коллаборативную фильтрацию. Выберем оценку схожести через *косинусную близость* и *item-based*-подход:

In [17]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}

knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


Посмотрим, какие рекомендации мы получили, с помощью следующей программы:

In [27]:
predictions = knn.test(testset)
print(predictions[:3])
print('\n'.join([str(prediction) for prediction in predictions[:3]]))


[Prediction(uid='7', iid='633', r_ui=5.0, est=4.199452349030111, details={'actual_k': 40, 'was_impossible': False}), Prediction(uid='422', iid='287', r_ui=3.0, est=3.4703437660463736, details={'actual_k': 40, 'was_impossible': False}), Prediction(uid='804', iid='163', r_ui=3.0, est=3.5716736533692854, details={'actual_k': 40, 'was_impossible': False})]
user: 7          item: 633        r_ui = 5.00   est = 4.20   {'actual_k': 40, 'was_impossible': False}
user: 422        item: 287        r_ui = 3.00   est = 3.47   {'actual_k': 40, 'was_impossible': False}
user: 804        item: 163        r_ui = 3.00   est = 3.57   {'actual_k': 40, 'was_impossible': False}


Информация о каждой паре будет содержать следующие характеристики:

- **uid** — id пользователя;
- **iid** — id элемента;
- **r_ui** (float) — реальный рейтинг, который этот пользователь поставил этому элементу;
- **est** (float) — предсказанный рейтинг.

## Задание 3.5

1. Каков реальный рейтинг, выставленный пользователем с ID 500 для фильма с ID 699?

2. Каков прогнозируемый рейтинг для пользователя с ID 500 и фильма с ID 699? Ответ округлите до двух знаков после точки-разделителя.

In [31]:
real_rating, predicted_rating = None, None

for prediction in predictions:
    if prediction.uid == '500' and prediction.iid == '699':
        real_rating = prediction.r_ui
        predicted_rating = round(prediction.est, 2)
        break

print("Реальный рейтинг пользователя с ID 500 для фильма с ID 699:", real_rating)
print("Прогнозируемый рейтинг для пользователя с ID 500 и фильма с ID 699:", predicted_rating)

Реальный рейтинг пользователя с ID 500 для фильма с ID 699: 3.0
Прогнозируемый рейтинг для пользователя с ID 500 и фильма с ID 699: 3.47


Вычислить RMSE для получившихся предсказаний:

In [43]:
rmse_score = accuracy.rmse(predictions)
print(f"RMSE оценка: {rmse_score:.2f}")

RMSE: 1.0272
RMSE оценка: 1.03


Итак, мы построили систему рекомендаций и даже оценили её качество. Но как же вывести рекомендации для конкретного пользователя?
Оформим наши предсказания в таблицу и отсортируем их по прогнозируемому рейтингу:

In [44]:
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)
pred.head()

,uid,iid,r_ui,est,details
22469,849,234,5.0,4.951929,"{'actual_k': 19, 'was_impossible': False}"
1974,849,427,4.0,4.950547,"{'actual_k': 19, 'was_impossible': False}"
8272,849,568,4.0,4.949215,"{'actual_k': 19, 'was_impossible': False}"
5138,849,174,5.0,4.947691,"{'actual_k': 19, 'was_impossible': False}"
22021,688,1127,5.0,4.928412,"{'actual_k': 15, 'was_impossible': False}"


Теперь мы можем вывести рекомендуемые для конкретного пользователя фильмы, начиная от наиболее релевантного (с точки зрения рекомендаций) и заканчивая наименее релевантным.

In [45]:
recom = pred[pred.uid =='849']['iid'].to_list()
recom

['234', '427', '568', '174']

## Задание 3.6

Реализуйте user-based-алгоритм. Какое значение **RMSE** получилось для коллаборативной фильтрации типа *user-based*? Ответ округлите до двух знаков после точки-разделителя.

In [46]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}
knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)
predictions = knn.test(testset)
accuracy.rmse(predictions).round(2)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0175


1.02

## Задание 3.7

Теперь давайте сравним полученные результаты с результатами **SVD-алгоритма**. Реализуйте SVD с параметрами по умолчанию.

Какое значение RMSE получилось для SVD? Ответ округлите до двух знаков после точки-разделителя.

In [47]:
model = SVD()
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions).round(2)

RMSE: 0.9411


0.94

# 4. Гибридные модели

In [3]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808328 sha256=3efd97b0a1308ac01b8a107ae3699cf52da96b61cc1f7b3e850cbd36199c20f3
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [4]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

In [5]:
from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/DS/')

ratings = pd.read_csv('data/ratings.csv') # Поставленные оценки
books = pd.read_csv('data/books.csv') # Информация о книгах
tags = pd.read_csv('data/tags.csv') # Информация о тегах
book_tags = pd.read_csv('data/book_tags.csv') # Книги с тегами

Mounted at /content/drive


Сначала посмотрим на набор данных books: в этих данных есть обычный id книги, а есть id книги в системе Goodreads — этот id отображён в признаке goodreads_book_id. В других данных (book_tags) указан только id книги в системе Goodreads, поэтому нам необходимо добавить туда обычный id.

In [6]:
books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


## Задание 4.1

Добавьте в набор данных book_tags признак с обычным id книги, используя соответствие обычного id и id в системе Goodreads.

Какой обычный id у книги, которая имеет id 5 в системе Goodreads?

In [7]:

book_tags = book_tags.merge(books[['book_id', 'goodreads_book_id']], on='goodreads_book_id', how='left')

ordinary_id = book_tags[book_tags['goodreads_book_id'] == 5]['book_id'].values[0]
print("Обычный id книги с Goodreads id 5:", ordinary_id)


Обычный id книги с Goodreads id 5: 18


In [8]:
book_tags.head()

,goodreads_book_id,tag_id,count,book_id
0,1,30574,167697,27
1,1,11305,37174,27
2,1,11557,34173,27
3,1,8717,12986,27
4,1,33114,12716,27


## Задание 4.2

Необходимо оставить в наборе данных book_tags только те записи, теги для которых есть в данных tags.

Отфильтруйте данные таким образом, чтобы в наборе данных book_tags остались только те строки, в которых находятся теги, информация о которых есть в наборе данных tags.

Сколько объектов осталось?

In [9]:
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags.shape

(300738, 4)

Отлично, мы подготовили информацию о тегах книг — это будет метаинформацией для построения рекомендательной системы. Теперь нам необходимо подготовить данные о взаимодействии пользователей и книг. Для этого нам понадобится файл ratings.

Оба набора данных (и про взаимодействия, и про метаинформацию) необходимо преобразовать в разрежённые матрицы. Это можно сделать с помощью специальной функции из модуля scipy:

In [10]:
!pip install scipy==1.10
from scipy.sparse import csr_matrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 31.3 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4


Нам важно преобразовать данные в специальный формат, в котором хранятся разрежённые матрицы — будем использовать формат Compressed Sparse Row (CSR), подразумевающий подсчёт кумулятивной суммы количества элементов в строке вместо индексов строк.

In [11]:
# Передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы),
# а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)
ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id,ratings.book_id)))

Теперь нам необходимо составить матрицу с метаданными. В качестве индексов будут выступать id книги и id тега, и если у этой книги есть рассматриваемый тег, то на пересечении соответствующих строки и столбца будет выставлена единица.

In [14]:
meta_matrix = csr_matrix(([1]*len(book_tags),(book_tags.book_id,book_tags.tag_id)))

## Задание 4.4

Давайте проверим, что всё получилось правильно.

Каково среднее арифметическое значений разрежённой матрицы с рейтингами? Ответ округлите до трёх знаков после точки-разделителя.

In [16]:
ratings_matrix.mean().round(3)

0.007

Отлично, данные подготовлены — теперь настало время определить модель, которую мы будем использовать. Сделаем это следующим образом:

In [17]:
model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

В качестве функции потерь мы выбрали значение *'warp'*, хотя, разумеется, это не единственный вариант. В модуле **LightFM** представлены следующие функции потерь:

- *'logistic'* — логистическая функция. Полезна в случаях, когда есть как положительные, так и отрицательные взаимодействия, например 1 и -1.
- *'bpr'* — байесовский персонализированный рейтинг. Можно применять, когда присутствуют только положительные взаимодействия.
- *'warp'* — парный взвешенный приблизительный ранг. Используется, если необходимо повысить качество именно в верхней части списка рекомендаций.
- *'warp-kos'* — модификация warp.

In [18]:
train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

Теперь обучим модель на наших данных о взаимодействии, также используя метаданные о книгах. Для этого воспользуемся методом fit(). В этот метод передадим обучающую выборку, признаки товаров — item_features, количество эпох обучения (сколько раз мы будем показывать модели исходный датасет, чтобы она лучше выучила данные) — epochs, а также параметр verbose для отслеживания процесса обучения:

In [19]:
model = model.fit(
    train, # Обучающая выборка
    item_features=meta_matrix, # Признаки товаров
    epochs=10, # Количество эпох
    num_threads=6, # for Google Colab
    verbose=True # Отображение обучения
)

Epoch: 100%|██████████| 10/10 [10:31<00:00, 63.13s/it]


## Задание 4.5

Оцените качество полученной модели с помощью функции precision_at_k, передав в неё три аргумента: модель, тестовые данные и обозначение метаданных (item_features = meta_matrix).

Примечание. Процесс расчёта метрик рекомендательной системы также является довольно затратным по времени. Для ускорения этого процесса вы можете передать параметр num_threads, чтобы указать количество потоков процессора, используемых для вычислений.

Выведите среднее арифметическое и округлите его до двух знаков после точки-разделителя.

In [20]:
prec_score = precision_at_k(
                model
                , test
                , num_threads=6 # for Google Colab
                , item_features = meta_matrix).mean()

print(prec_score)

0.023665428


В рекомендательных системах метрики интерпретируются иначе, чем в задачах классификации. Показатели точности РС считаются хорошими, если они находятся в районе 0.1-0.3.

У нас получился не слишком высокий, но довольно неплохой результат. Чтобы его улучшить, можно попробовать следующее:

Поработать над предобработкой данных, добавив в них дополнительную информацию о товарах. Также можно попробовать воспользоваться иным способом создания разреженной матрицы, например, форматом coo_matrix() или csc_matrix(), которые также входят в библиотеку scipy. Подробнее почитать о них вы можете здесь.
Поиграться с параметрами модели LightFM — поуправлять темпом обучения (learning_rate), размерностью вектора для представления (no_components), количеством эпох обучения (epochs) и функцией потерь (loss).

Для предсказания рейтинга для нового пользователя можно воспользоваться методом predict():

In [ ]:
scores = model.predict(<индекс интересующего пользователя>, np.arange(n_items),user_features=new_user_feature)